# Redes Neuronales

# Problema: Clasificación de Imágenes de Aves de India

Se tiene un data set que contiene 15.000 imágenes de aves comunes de India, las cuales se dividen en 25 categorías distintas (600 imágenes cada categoría). Adicionalmente, cada categoría se ha dividido en 2 grupos donde 550 imágenes pertenecen a `/train` y 50 imágenes pertenencen a `/valid`. Es decir que de las 15.000 imágenes 13.750 son para entrenamiento y 1250 para validacion.

Se desea realizar una red neuronal que permita clasificar estas 25 especies de aves y para ello, utilice la técnica de Transferencia de Aprendizaje (Transfer Learning) para utilizar una red previamente entrenada y desarrollar un nuevo modelo.

Las imágenes y el modelo entrenado se encuentran en el siguiente link: https://drive.google.com/drive/folders/1p35rHHzd3jahMoY5hh1l-tA_AHI2PXG_?usp=sharing



Este archivo contiene la siguiente estructura

- filtered_training_set
    - train
        - Asian Green Bee-Eater
        - Brown-Headed Barbet
        - **...**
        - White-Breasted Waterhen
    - valid
        - Asian Green Bee-Eater
        - Brown-Headed Barbet
        - **...**
        - White-Breasted Waterhen
        
Las categorías de cada imagen se encuentran según la carpeta donde estén alojadas las imágenes.

## El modelo que usaremos para la transferencia de aprendizaje es: InceptionV3

## Iniciamos con el procesamiento de datos

In [1]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import matplotlib.pyplot as plt
import pathlib, os, random
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential

## Path de dataset

In [2]:
train_dir = 'filtered_training_set/train'  
val_dir = 'filtered_training_set/valid'

## Generador de imágenes (entrenamiento y validación)

In [3]:
# Rescale
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)
valid_datagen = ImageDataGenerator(rescale = 1./255)

# data transfer from directories to batches
train_data = train_datagen.flow_from_directory(directory = train_dir,
                                               batch_size= 32,
                                               target_size= (224,224),
                                               class_mode = "categorical")

val_data = valid_datagen.flow_from_directory(directory = val_dir,
                                               batch_size = 32,
                                               target_size = (224,224),
                                               class_mode = "categorical")

Found 13750 images belonging to 25 classes.
Found 1250 images belonging to 25 classes.


## Instancia del modelo base

In [4]:
base_model = tf.keras.applications.InceptionV3(include_top= False,input_shape=(224, 224, 3))

base_model.trainable = False

inputs = tf.keras.layers.Input(shape =(224, 224,3), name = "input-layer")

x = base_model(inputs)
x = tf.keras.layers.GlobalAveragePooling2D(name = "global_average_pooling_layer")(x)
outputs = tf.keras.layers.Dense(25, activation = "softmax", name = "output-layer")(x)
model = tf.keras.Model(inputs, outputs)
model.compile(loss = "categorical_crossentropy", optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01), metrics = ["accuracy"])

history = model.fit(train_data,
                                 epochs=10,
                                 steps_per_epoch = len(train_data),
                                 validation_data = val_data,
                                 validation_steps = int(0.25*len(val_data)),)

Epoch 1/10
430/430 [==============================] - 771s 2s/step - loss: 1.7988 - accuracy: 0.7263 - val_loss: 2.0553 - val_accuracy: 0.7094
Epoch 2/10
430/430 [==============================] - 826s 2s/step - loss: 1.2646 - accuracy: 0.8285 - val_loss: 1.8785 - val_accuracy: 0.8031
Epoch 3/10
430/430 [==============================] - 875s 2s/step - loss: 1.0537 - accuracy: 0.8563 - val_loss: 2.0060 - val_accuracy: 0.7937
Epoch 4/10
430/430 [==============================] - 875s 2s/step - loss: 0.8737 - accuracy: 0.8841 - val_loss: 1.7840 - val_accuracy: 0.8438
Epoch 5/10
430/430 [==============================] - 909s 2s/step - loss: 0.8074 - accuracy: 0.8943 - val_loss: 3.3281 - val_accuracy: 0.8000
Epoch 6/10
430/430 [==============================] - 910s 2s/step - loss: 0.8549 - accuracy: 0.8969 - val_loss: 2.5517 - val_accuracy: 0.8094
Epoch 7/10
430/430 [==============================] - 912s 2s/step - loss: 0.6285 - accuracy: 0.9173 - val_loss: 2.8645 - val_accuracy: 0.8344

## Resumen del modelo

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input-layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 global_average_pooling_laye  (None, 2048)             0         
 r (GlobalAveragePooling2D)                                      
                                                                 
 output-layer (Dense)        (None, 25)                51225     
                                                                 
Total params: 21,854,009
Trainable params: 51,225
Non-trainable params: 21,802,784
_________________________________________________________________


## Congelamiento de los top layers para Fine-Tunning

In [6]:
base_model.trainable = True

# Un-freeze last 10 layers
for layer in base_model.layers[:-10]:
  layer.trainable = False

# Recompile (we have to compile model every time there is a change)
model.compile(loss = "categorical_crossentropy",
                optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), # when fine-tuning you typically want to lower lr by 10x
                 metrics = ["accuracy"] )

for layer_number, layer in enumerate(model.layers[1].layers):
  print(layer_number, layer.name, layer.trainable)

print(len(model.trainable_variables))

0 input_1 False
1 conv2d False
2 batch_normalization False
3 activation False
4 conv2d_1 False
5 batch_normalization_1 False
6 activation_1 False
7 conv2d_2 False
8 batch_normalization_2 False
9 activation_2 False
10 max_pooling2d False
11 conv2d_3 False
12 batch_normalization_3 False
13 activation_3 False
14 conv2d_4 False
15 batch_normalization_4 False
16 activation_4 False
17 max_pooling2d_1 False
18 conv2d_8 False
19 batch_normalization_8 False
20 activation_8 False
21 conv2d_6 False
22 conv2d_9 False
23 batch_normalization_6 False
24 batch_normalization_9 False
25 activation_6 False
26 activation_9 False
27 average_pooling2d False
28 conv2d_5 False
29 conv2d_7 False
30 conv2d_10 False
31 conv2d_11 False
32 batch_normalization_5 False
33 batch_normalization_7 False
34 batch_normalization_10 False
35 batch_normalization_11 False
36 activation_5 False
37 activation_7 False
38 activation_10 False
39 activation_11 False
40 mixed0 False
41 conv2d_15 False
42 batch_normalization_15 False

## Fine-Tunning

In [7]:
initial_epochs = 10
fine_tune_epochs = initial_epochs + 1
history_2 = model.fit(train_data,
                       epochs = fine_tune_epochs,
                       validation_data = val_data,
                       validation_steps = int(0.25*len(val_data)),
                       initial_epoch =  history.epoch[-1],) # Start the epoch where it left before

Epoch 10/11
430/430 [==============================] - 848s 2s/step - loss: 0.1018 - accuracy: 0.9797 - val_loss: 2.0766 - val_accuracy: 0.8813
Epoch 11/11
430/430 [==============================] - 824s 2s/step - loss: 0.0481 - accuracy: 0.9882 - val_loss: 2.9486 - val_accuracy: 0.8219


In [8]:
model.save('birdman.h5')